In [1]:
!pip install spacy

In [2]:
!pip install spacy-entity-linker

  Preparing metadata (setup.py) ... done
  Created wheel for spacy-entity-linker: filename=spacy_entity_linker-1.0.3-py3-none-any.whl size=14375 sha256=67f0b1d9ec9dae1ec309426a45a3cd29d47f9c0c5b1b3c998e2c072a73ade7bf
  Stored in directory: /root/.cache/pip/wheels/6d/73/f9/568b5f541ed2d6f9adb8556b8ef9995e5528f3f9ab2b83b196
Successfully built spacy-entity-linker


In [3]:
#!python -m spacy download en_core_web_md
# Tester le modèle en anglais également
!python -m spacy download fr_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 14.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import json
import spacy
import glob
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Définition des fonctions lire_fichier et stocker (identiques à votre version)
def lire_fichier(chemin):
    try:
        with open(chemin, "r", encoding="utf-8") as fichier:
            texte = fichier.read()
            return texte
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {chemin}: {e}")
        return None

def stocker(chemin, contenu):
    try:
        with open(chemin, "w", encoding="utf-8") as w:
            json.dump(contenu, w, ensure_ascii=False, indent=2)
    except Exception as e:
        print(f"Erreur lors de l'écriture du fichier {chemin}: {e}")

In [7]:
# Initialisation de la variable modele
modele = "fr_core_news_md" #à modifier en fonction du modèle utilsé
outil_utilise = "spacy-entitylinking"

# Chemins de sortie
output_dir = "/content/drive/MyDrive/Colab Notebooks/Stage/LiageTest/SORTIEJSON_LINKING"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"Pour le modèle : {modele}\n")
nlp = spacy.load(modele)

# Ajouter le pipeline EntityLinker si nécessaire
if "entityLinker" not in nlp.pipe_names:
    nlp.add_pipe("entityLinker", last=True)

# Path vers les fichiers à traiter
path_corpora = "/content/drive/MyDrive/Colab Notebooks/Stage/LiageTest/TEST/*.txt"

# Parcourir tous les fichiers correspondant au pattern path_corpora
for path in glob.glob(path_corpora):
    print("Entrée : ", path)
    txt = lire_fichier(path)
    if txt is None:
        continue

    doc = nlp(txt)
    dico_combined = {}

    i = 0
    for ent in doc.ents:
        span = ent
        ide = f"ID{i}"
        i += 1

        start_char = span.start_char
        end_char = span.end_char
        contexte_entite = txt[max(start_char - 200, 0):min(end_char + 200, len(txt))]

        if ide not in dico_combined:
            dico_combined[ide] = {
                "contexte": contexte_entite,
                "entite": span.text,
                "type_entite": span.label_,
                "entite_associee": []  # Initialisation de la liste d'entités associées
            }

        if span._.linkedEntities:
            for linked_entity in span._.linkedEntities:
                url = linked_entity.get_url()
                label = linked_entity.get_label()
                description = linked_entity.get_description()

                dico_combined[ide]["entite_associee"].append({
                    "kb_id": url,
                    "label": label,
                    "text": span.text,  # Texte d'entrée correspondant à l'entité
                    "description": description
                })

    output_filename = f"{os.path.splitext(os.path.basename(path))[0]}_{modele}_{outil_utilise}.json"
    output_path_combined = os.path.join(output_dir, output_filename)
    stocker(output_path_combined, dico_combined)
    print(f"Fichier combiné enregistré avec succès : {output_path_combined}")

Pour le modèle : fr_core_news_md

Entrée :  /content/drive/MyDrive/Colab Notebooks/Stage/LiageTest/TEST/ADAM_Mon-village_REF.txt


Fichier combiné enregistré avec succès : /content/drive/MyDrive/Colab Notebooks/Stage/LiageTest/SORTIEJSON_LINKING/ADAM_Mon-village_REF_combined-fr_core_news_md-spacy-entitylinking.json


In [ ]:
# Transformation des données

In [10]:
# Définition des fonctions lire_fichier_json et stocker_json
def lire_fichier_json(chemin):
    try:
        with open(chemin, "r", encoding="utf-8") as fichier:
            contenu = json.load(fichier)
            return contenu
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {chemin}: {e}")
        return None

def stocker_json(chemin, contenu):
    try:
        with open(chemin, "w", encoding="utf-8") as w:
            json.dump(contenu, w, ensure_ascii=False, indent=2)
    except Exception as e:
        print(f"Erreur lors de l'écriture du fichier {chemin}: {e}")

def transformer_donnees(entrees, nlp):
    resultats = {}
    for ide, valeurs in entrees.items():
        entite = valeurs["entite"]
        contexte_entite = valeurs["contexte"]
        type_entite = valeurs["type_entite"]
        entite_associee = valeurs["entite_associee"]

        # Convertir l'entité en un document SpaCy
        doc_entite = nlp(entite)
        linked_entities = []

        # Calculer la similarité entre l'entité et chaque entité associée
        for ent in entite_associee:
            # Convertir l'entité associée en un document SpaCy
            doc_linked = nlp(ent["label"])
            # Calculer la similarité cosinus entre les vecteurs des deux documents
            score = doc_entite.similarity(doc_linked)
            linked_entities.append({
                "kb_id": ent["kb_id"],
                "label": ent["label"],
                "text": ent["text"],  # Texte d'entrée correspondant à l'entité
                "description": ent["description"],
                "score": score
            })

        resultats[ide] = {
            "contexte": contexte_entite,
            "entite": entite,
            "type_entite": type_entite,
            "entite_associee": linked_entities
        }
    return resultats

In [11]:
# Initialisation de la variable modele
modele = "fr_core_news_md" #à modifier en fonction du modèle de langue utilisé
nlp = spacy.load(modele)

# Chemins de sortie
output_dir = "/content/drive/MyDrive/Colab Notebooks/Stage/LiageTest/SORTIEJSON_LINKING"
transformed_output_dir = "/content/drive/MyDrive/Colab Notebooks/Stage/LiageTest/Resultat_entity-linking"

# Créer le dossier pour stocker les fichiers transformés
if not os.path.exists(transformed_output_dir):
    os.makedirs(transformed_output_dir)

# Parcourir tous les fichiers JSON dans le dossier output_dir
for json_file in glob.glob(os.path.join(output_dir, "*.json")):
    print(f"Traitement du fichier : {json_file}")

    # Lire le fichier JSON d'entrée
    donnees_entrees = lire_fichier_json(json_file)

    if donnees_entrees is None:
        continue

    # Transformer les données
    donnees_transformees = transformer_donnees(donnees_entrees, nlp)

    # Générer le nom de fichier de sortie
    base_name = os.path.splitext(os.path.basename(json_file))[0]
    output_path = os.path.join(transformed_output_dir, f"{base_name}_transformed.json")

    # Stocker les données transformées dans un nouveau fichier JSON
    stocker_json(output_path, donnees_transformees)
    print(f"Fichier transformé enregistré avec succès : {output_path}")

Traitement du fichier : /content/drive/MyDrive/Colab Notebooks/Stage/LiageTest/SORTIEJSON_LINKING/ADAM_Mon-village_REF_combined-fr_core_news_md-spacy-entitylinking.json


<ipython-input-10-3af108d160b7>:35: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  score = doc_entite.similarity(doc_linked)


Fichier transformé enregistré avec succès : /content/drive/MyDrive/Colab Notebooks/Stage/LiageTest/Resultat_entity-linking/ADAM_Mon-village_REF_combined-fr_core_news_md-spacy-entitylinking_transformed.json
